# Installs
Install, then `restart session` if Import is unsuccessful

In [1]:
%pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
!pip install -U bitsandbytes

# Imports

In [3]:
import os
import pandas as pd
import json
import time
import re

# openAI
from openai import OpenAI

# llama
from transformers import AutoModelForCausalLM, AutoTokenizer

# drive
from google.colab import drive
from google.colab import userdata

# stats
from scipy.stats import spearmanr, pearsonr



# Setup

In [6]:
# Google Drive Setup

# mount google drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Filepaths

In [7]:
question_path = '/content/drive/MyDrive/Colab Notebooks/final_project_266/context_question_answer_only.csv' # 50 test questions
question_path = '/content/drive/MyDrive/Colab Notebooks/final_project_266/200_questions.csv' # 200 test questions


In [8]:
question_context_df = pd.read_csv(question_path)
question_context_df = question_context_df[['context', 'question']]
question_context_df

,context,question
0,"When you turn over the pages, you are actually...",What is the role of the Sadhaka in the context...
1,"You have the unique chance of meeting Him, at ...",What implication does the passage have for the...
2,"In the pages of the magazine, published with H...",What challenge does the passage identify in th...
3,Until one gets firmly fixed in the path that g...,What is the significance of attuning the mind ...
4,When Ignorance and its concomitant Delusion di...,In what way does deep sleep affect the percept...
...,...,...
200,"Through inquiry on these lines, knowledge is r...",What is the significance of inquiry in achievi...
201,It cannot be got in Swarga (heaven) or in Moun...,"According to the passage, where can true knowl..."
202,He hasachieved it by dwelling on the Mithya (u...,"What does the term ""Mithya"" refer to in the pa..."
203,The Jivanmuktha has no trace of the “will toli...,What transformation occurs in the aspirant's u...


## Load the llama 8b model

Load the pre-trained Llama 8B model and tokenizer.


In [9]:

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_8bit=True)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [10]:
from huggingface_hub import notebook_login

notebook_login()

## Define Prompt


In [11]:
prompt_template = """
Use the following context to answer the question. Only use the information in the context.

Context: {context}

Question: {question}

Answer:
"""

## Implement answer generation



In [12]:
def generate_answer(context, question):
  """
  Generates an answer to a question based on the provided context using the Llama model.

  Args:
    context: The context passage (string).
    question: The question to answer (string).

  Returns:
    The generated answer string.
  """
  formatted_prompt = prompt_template.format(context=context, question=question)
  input_ids = tokenizer(formatted_prompt, return_tensors="pt").input_ids.to(model.device)

  # Generate the answer. Adjust max_new_tokens as needed.
  generated_ids = model.generate(input_ids, max_new_tokens=150, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

  # Decode the generated tokens, skipping the prompt tokens
  answer = tokenizer.decode(generated_ids[0, input_ids.shape[1]:], skip_special_tokens=True)

  return answer

## Run Model


In [14]:
question_context_df.head()
question_context_df.tail()

,context,question
200,"Through inquiry on these lines, knowledge is r...",What is the significance of inquiry in achievi...
201,It cannot be got in Swarga (heaven) or in Moun...,"According to the passage, where can true knowl..."
202,He hasachieved it by dwelling on the Mithya (u...,"What does the term ""Mithya"" refer to in the pa..."
203,The Jivanmuktha has no trace of the “will toli...,What transformation occurs in the aspirant's u...
204,Hewill not mistake the rope for the snake. He ...,"What is the significance of the term ""Abhasa A..."


In [16]:
# Select a few rows for testing
# test_cases = question_context_df.sample(5, random_state=42)

gen_answer = []

# Iterate and test the generate_answer function
n = 0
for index, row in question_context_df.iterrows():
  context = row['context']
  question = row['question']

  # print(f"Context:\n{context}\n")
  # print(f"Question:\n{question}\n")

  try:
    answer = generate_answer(context, question)
    gen_answer.append(answer)
    print(f"Generated Answer:\n{answer}\n")
  except Exception as e:
    print(f"Error generating answer: {e}\n")

  n += 1
  print(index)

  print("-" * 50)

Generated Answer:
The Sadhaka is a seeker of spiritual knowledge and guidance. Bhagavan Sri Sathya Sai Baba is guiding them and granting them peace and perfection. The Sadhaka is exhorted to start on the spiritual pilgrimage this very day, implying that they are expected to take action and follow the guidance of Bhagavan Sri Sathya Sai Baba. The Sadhaka is also expected to place all their burdens on Bhagavan Sri Sathya Sai Baba, indicating that they are to surrender their problems and worries to Him. Overall, the role of the Sadhaka is to seek guidance and follow the teachings of Bhagavan Sri Sathya Sai Baba in order to achieve spiritual growth and perfection.

0
--------------------------------------------------
Generated Answer:
The passage does not have any implication for the pursuit of worldly desires. It is focused on the pursuit of spiritual growth and the meeting with Him (possibly referring to Bhagavan Sri Sathya Sai Baba) at Prasanthi Nilayam. The passage emphasizes the impor

In [17]:
llama_generated_answers = question_context_df.copy()
llama_generated_answers['generated_answer'] = gen_answer
llama_generated_answers.tail()

,context,question,generated_answer
200,"Through inquiry on these lines, knowledge is r...",What is the significance of inquiry in achievi...,"According to the passage, inquiry on the lines..."
201,It cannot be got in Swarga (heaven) or in Moun...,"According to the passage, where can true knowl...","According to the passage, true knowledge of th..."
202,He hasachieved it by dwelling on the Mithya (u...,"What does the term ""Mithya"" refer to in the pa...","The term ""Mithya"" refers to the unreality of t..."
203,The Jivanmuktha has no trace of the “will toli...,What transformation occurs in the aspirant's u...,The aspirant's understanding transforms in tha...
204,Hewill not mistake the rope for the snake. He ...,"What is the significance of the term ""Abhasa A...","The term ""Abhasa Avaranam"" refers to the mista..."


In [18]:
llama_generated_answers.to_csv('/content/drive/MyDrive/Colab Notebooks/final_project_266/llama_control_generated_answers_200.csv', index=False)
